# Data Wrangling: Join, Combine and Reshape
El principal cometido de este apartado es combinar, mezclar, juntar DataFrames.

# Índice

0. Load Pandas


1. Hierarchical Indexing

    1.1 Series
    
        1.1.1 Partial Indexing and Slicing
        
        1.1.2 Rearrange to DataFrame
        
    1.2 DataFrame
    
        1.2.1 Partial Indexing and Slicing
        
        1.2.2 Names to Hierarchical Levels
    
    1.3 Reordering and Sorting Levels
    
    1.4 Summary Statistics by Level
    
    1.5 Indexing with a DataFrame's columns
    
        1.5.1 set_index() Method
        
        1.5.2 reset_index() Method
       
    
2. Combining and Merging Datasets

    2.1 Merge 
    
        2.1.1 Merging with a single key
    
        2.1.2 Merging with multiple keys
        
        2.1.3 Overlapping Column Names
    
    2.2 Merging on Index
    
        2.2.1 Hierarchically Indexed Data
    
        2.2.2 Merging Indexes on Both Sides
        
    2.3 Concatenating Along an Axis
    
        2.3.1 Concatenation
    
        2.3.2 Identifiying the Concatenation
        
        2.3.3 Naming the Levels
        
        2.3.4 Reseting the Index
        
        
3. Reshaping

    3.1 Melt
    
    3.2 Stack and Unstack
    
        3.2.1 Handiling with NaN
        
    3.3 Pivoting

# Load Data

In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

# 1. Hierarchical Indexing
Podemos crear Series y DataFrames que contengan 2 o más index en distintos subniveles.

Hierarchical Indexing nos permite trabajar con múltiples (2 o +) index levels en un axis. 

## 1.1 Series

Creamos una Serie con MultiIndex:

In [2]:
data = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1    0.607015
   2   -0.117938
   3   -0.947302
b  1   -1.215998
   3    0.747627
c  1    0.796048
   2    1.191556
d  2   -0.003209
   3   -0.870326
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

### 1.1.1 Partial Indexing and Slicing
Podemos seleccionar subcojuntos:

In [4]:
data["b"]

1   -1.215998
3    0.747627
dtype: float64

In [5]:
data["b":"c"]

b  1   -1.215998
   3    0.747627
c  1    0.796048
   2    1.191556
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1   -1.215998
   3    0.747627
d  2   -0.003209
   3   -0.870326
dtype: float64

In [7]:
data.iloc[0:3]

a  1    0.607015
   2   -0.117938
   3   -0.947302
dtype: float64

In [8]:
data.loc[:,2]

a   -0.117938
c    1.191556
d   -0.003209
dtype: float64

### 1.1.2 Rearrange to DataFrame
Podemos reorganizar los datos de la Serie y convertirlos en un DataFrame 
    
#### unstack Method

In [9]:
data.unstack()

,1,2,3
a,0.607015,-0.117938,-0.947302
b,-1.215998,NaN,0.747627
c,0.796048,1.191556,NaN
d,NaN,-0.003209,-0.870326


La operación inversa sería hacer stack() Method

In [10]:
data.unstack().stack()

a  1    0.607015
   2   -0.117938
   3   -0.947302
b  1   -1.215998
   3    0.747627
c  1    0.796048
   2    1.191556
d  2   -0.003209
   3   -0.870326
dtype: float64

## 1.2 DataFrame
Cada columna (y fila) puede tener también un hierarchical index:

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

### 1.2.1 Partial Indexing and Slicing
Seleccionar subconjuntos en columnas funciona similar a cómo se seleccionan en filas

In [12]:
frame["Ohio"]

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

### 1.2.2 Names to Hierarchical Levels
Podemos dar un nombre a los distintos niveles que componen el DataFrame

In [13]:
frame.index.names = ["key1", "key2"]

frame.columns.names = ["state", "color"]

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## 1.3 Reordering and Sorting Levels
En ocasiones queremos reorganizar los niveles de los axis internos u organizar los datos en base a los datos de un nivel en específico.

#### swaplevel()
swaplevel() nos permite seleccionar dos niveles (números o nombres) y nos devuelve un nuevo objeto con los niveles intercambiados (pero los datos inalterados)

In [14]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

#### sort_index()
Ordena los datos usando sólo los valores de un sólo nivel y lo ordena alfabéticamente según el nivel seleccionado

En este ejemplo, se ordenan teniendo en cuenta el nivel 1 (es decir, el correspondiente a la columna key2):

In [15]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## 1.4 Summary Statistics by Level
Podemos realizar operaciones matemáticas y estadísticas entre niveles

In [17]:
frame.sum(level = "key2")

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level = "key1")

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

## 1.5 Indexing with a DataFrame's columns
Podemos utilizar las columnas como un row index y viceversa.

In [19]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two',
                                                                 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### 1.5.1 set_index() Method
Nos permite crear un nuevo DataFrame usando una o más columnas como row index

In [20]:
frame2 = frame.set_index(["c","d"])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

### 1.5.1 reset_index() Method
---Nos permite usar las filas (row index) como columnas

Elimina niveles y reasigna un index desde 0

In [21]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 2. Combining and Merging Datasets
Los datos contenidos en diferentes DataFrame pueden combinarse/ fusionarse.

## 2.1 Merge
Nos permite combinar/ mezclar DataFrames en base a los valores de las columnas que tienen los DataFrames en común.

Juntando filas usando una o más keys (poniendo filas unas encima de las otras)

In [22]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/875/1*-uSHoxrzM57syqnKnms2iA.png", width = 600)

### 2.1.1 Merging with a single key

In [23]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [24]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


#### merging dataframes with common columns
Cuando tenemos 2 DataFrame pero comparten la "merge key".

Por defecto, how = "inner" ---> inersección

Por defecto, on = the common one si no se especifica otra cosa

Intersección:

In [26]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [27]:
pd.merge(df1,df2, on = "key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Unión:

In [28]:
pd.merge(df1,df2, on = "key", how = "outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### merging dataframes with different columns
Cuando tenemos 2 DataFrames con una "merge key" diferente cada uno 

Tenemos que especificarlas por separado

Hacemos uso de left_on = "nombre columna a la izquierda", right_on = "nombre columna a la derecha"

In [29]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [30]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [31]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [32]:
pd.merge(df3,df4, left_on = "lkey", right_on = "rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [33]:
Image(url= "https://i.gyazo.com/7eca09806d6f1197a14cfcc0ffe86005.png", width = 400)

### 2.1.2 Merging with multiple keys
Cuando uno de los DataFrame tiene 2 o + "merge keys"

In [34]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [35]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [36]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [37]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


### 2.1.3 Overlapping Column Names
Cuando hay una o varias columnas que tienen el mismo nombre (y no son las que se usan para hacer merge) pandas genera automáticamente unos sufijos para diferenciarlos.

Nosotros podemos poner el sufijo que queramos gracias a suffixes = ()

In [38]:
pd.merge(left, right, on = "key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### suffixes()

In [39]:
pd.merge(left, right, on = "key1", suffixes = ("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [40]:
Image(url= "https://i.gyazo.com/49df6ea0a26b8f9a927e3f03052b2dca.png", width =600)

## 2.2 Merging on Index
En ocasiones, queremos combinar "keys" que se encuentran en los row index.

Para indicar el indez que tieen que utilizarse como merge key, podemos usar:

left_index = True 

right_index = True

In [41]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [42]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [43]:
right1

,group_val
a,3.5
b,7.0


In [44]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Explicación: con left_on, seleccionamos nuestra merge key, que será la columna "key", del dataframe seleccionado a la izquierda (left1). Con right_index = True, seleccionamos el dataframe de la derecha (right1) como el dataframe al que queremos hacer merge on index.

In [45]:
pd.merge(left1, right1, left_on='key', right_index=True, how = "outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### 2.2.1 Hierarchically Indexed Data

In [46]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                                                            [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])

In [47]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [48]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [49]:
pd.merge(lefth, righth, left_on = ["key1", "key2"], right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [50]:
pd.merge(lefth, righth, left_on = ["key1", "key2"], right_index = True, how = "outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


### 2.2.2 Merging Indexes on Both Sides

#### using indexes on both sides

In [51]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [52]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [53]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [54]:
pd.merge(left2, right2, how = "outer", left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### join() for merging by index

In [55]:
left2.join(right2, how = "outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


## 2.3 Concatenating Along an Axis
Concatenar nos permite juntar dataframes a lo largo de filas o columnas. Es una forma de stackear múltiples dataframes.


Nos permite juntar los index de varios datasets

In [56]:
Image(url= "https://miro.medium.com/max/875/1*0wu6DunCzPC4o9FIyRTW4w.png", width = 600)

### 2.3.1 Concatenation

In [57]:
s1 = pd.Series([0, 1], index=['a', 'b'])

s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

s3 = pd.Series([5, 6], index=['f', 'g'])

In [58]:
s1

a    0
b    1
dtype: int64

In [59]:
s2

c    2
d    3
e    4
dtype: int64

In [60]:
s3

f    5
g    6
dtype: int64

#### concat() nos permite concatenar

Por defecto, concatenar funciona en el axis = 0 (filas)

In [61]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

Podemos utilizar axis = 1 para concatenar por columnas (el resultado es un DataFrame)

In [62]:
pd.concat([s1,s2,s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Por defecto, siempre se ejecuta la unión join = "outer", pero podemos hacer la intersección con join = "inner"

In [63]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [64]:
pd.concat([s1,s4], axis = 1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [65]:
pd.concat([s1,s4], axis = 1, join = "inner")

,0,1
a,0,0
b,1,1


### 2.3.2 Identifiying the Concatenation
Ya que cuando concatenamos y obtenemos el resultado, no sabemos de donde procede cada concatenación. Podemos utilizar el argumento keys para diferenciar los niveles

Utilizamos el argumento keys

#### Series:

In [66]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [67]:
result = pd.concat([s1,s2,s3], keys = ["one", "two", "three"])

result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

#### Series to DataFrame:

In [68]:
pd.concat([s1,s2,s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [69]:
pd.concat([s1,s2,s3], axis = 1, keys = ["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### DataFrame:

In [70]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [71]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [72]:
df2

,three,four
a,5,6
c,7,8


In [73]:
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [74]:
pd.concat([df1, df2], keys=['level1', 'level2'])

one  two  three  four
level1 a  0.0  1.0    NaN   NaN
       b  2.0  3.0    NaN   NaN
       c  4.0  5.0    NaN   NaN
level2 a  NaN  NaN    5.0   6.0
       c  NaN  NaN    7.0   8.0

In [75]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

#### dictionaries
podemos pasar un diccionario para realizar la concatenación. Las keys del diccionario serán las utilizadas para el argumento keys.

In [76]:
pd.concat({'level1': df1, 'level2': df2}, axis = 1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

### 2.3.3 Naming the Levels
Podemos dar nombre a los niveles

Utilizamos el argumento: names

In [77]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

### 2.3.4 Reseting the Index
Si los index son irrelevantes, podemos resetearlos y ordenarlos desde 0

Utilizamos el argumento: ignore_index = True

In [78]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [79]:
df1

,a,b,c,d
0,-0.415575,-0.026632,2.813486,0.030724
1,0.617561,-0.362876,-0.631209,0.705689
2,0.873446,-0.266577,-1.359497,0.448058


In [80]:
df2

,b,d,a
0,0.618471,0.025450,0.056546
1,0.785845,-1.056525,-0.635674


In [81]:
pd.concat([df1,df2])

,a,b,c,d
0,-0.415575,-0.026632,2.813486,0.030724
1,0.617561,-0.362876,-0.631209,0.705689
2,0.873446,-0.266577,-1.359497,0.448058
0,0.056546,0.618471,NaN,0.025450
1,-0.635674,0.785845,NaN,-1.056525


In [82]:
pd.concat([df1,df2], ignore_index = True)

,a,b,c,d
0,-0.415575,-0.026632,2.813486,0.030724
1,0.617561,-0.362876,-0.631209,0.705689
2,0.873446,-0.266577,-1.359497,0.448058
3,0.056546,0.618471,NaN,0.025450
4,-0.635674,0.785845,NaN,-1.056525


In [83]:
Image(url= "https://i.gyazo.com/43d76b52af99f5a2f590cc565b8a4e65.png", width =600)

In [84]:
Image(url= "https://i.gyazo.com/b236c3d6ac853fecef414a355ec37bfc.png", width =600)

# 3. Reshaping
Podemos cambiar las filas por columnas y viceversa.

Existen varios métodos para hacer esto.

## 3.1 Melt

Melt se utiliza para convertir wide dataframes (anchos) en unos más estrechos. Es decir, un dataframe con muchas columnas puede transformarse en uno con menos columnas si convertimos algunas de estas columnas en filas.

In [85]:
df1 = pd.DataFrame({'city':['A','B','C'],
                   'day1':[22,25,28],
                   'day2':[10,14,13],
                   'day3':[25,22,26],
                   'day4':[18,15,17],
                   'day5':[12,14,18]})

df1

,city,day1,day2,day3,day4,day5
0,A,22,10,25,18,12
1,B,25,14,22,15,14
2,C,28,13,26,17,18


Explicación: tenemos 3 ciudades diferentes y las medidas de temperatura de 5 días distintos. Podemos interpretar los días como filas en una columna:

In [86]:
pd.melt(df1, ["city"])

,city,variable,value
0,A,day1,22
1,B,day1,25
2,C,day1,28
3,A,day2,10
4,B,day2,14
5,C,day2,13
6,A,day3,25
7,B,day3,22
8,C,day3,26
9,A,day4,18


In [87]:
melted = df1.melt(id_vars=['city'])
melted

,city,variable,value
0,A,day1,22
1,B,day1,25
2,C,day1,28
3,A,day2,10
4,B,day2,14
5,C,day2,13
6,A,day3,25
7,B,day3,22
8,C,day3,26
9,A,day4,18


#### pivot() and reset_index()
Nos permite dejar el dataframe como estaba inicialmente

In [88]:
reshaped = melted.pivot("city", "variable", "value")

reshaped

variable,day1,day2,day3,day4,day5
city,,,,,
A,22,10,25,18,12
B,25,14,22,15,14
C,28,13,26,17,18


Dado que al utilizar pivot() los row labels pasan a ser index, utilizamos reset_index() para eliminar estos row labels como index:

In [89]:
reshaped.reset_index()

variable,city,day1,day2,day3,day4,day5
0,A,22,10,25,18,12
1,B,25,14,22,15,14
2,C,28,13,26,17,18


#### var_name and value_name
Nos permite cambiarle el nombre a las columnas 

Explicación: queremos cambiar el nombre a las columnas creadas con melt() y, además, queremos ordenar por ciudades y resetear los index

In [90]:
df1.melt(id_vars=['city'], var_name = 'date', value_name = 'temperature').sort_values(by='city').reset_index(drop=True)

,city,date,temperature
0,A,day1,22
1,A,day2,10
2,A,day3,25
3,A,day4,18
4,A,day5,12
5,B,day1,25
6,B,day2,14
7,B,day3,22
8,B,day4,15
9,B,day5,14


## 3.2 Stack and Unstack
La función stack() incrementa los index level del dataframe.

Suele utilizarse en dataframes que contienen muchos multi-level index.

In [91]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### stack() 
Nos permite rotar desde columnas hacia filas, incluyéndolas en un nivel.

En este ejemplo, al haber 2 filas y 3 columnas, stack() nos devolverá un dataframe con 3x2 = 6 filas:

In [92]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

Ahora las columnas "one", "two", "three" pertenecen a otro nivel, incluyéndolas en el multi-level index.

In [93]:
data.shape

(2, 3)

In [94]:
result.shape

(6,)

In [95]:
result.index[0] # multivel index

('Ohio', 'one')

In [96]:
result.index[3] # multivel index

('Colorado', 'one')

In [97]:
len(result.index.levels) # número de levels

2

#### unstack() 
Sirve para pivotar deshacer niveles, es decir, deshacer el stack. Por defecto, se hace unstack al nivel más profundo (más a la derecha).

In [113]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [98]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Podemos hacer unstack() a diferentes niveles pasando un número (el número del nivel) o nombre (nombre del nivel)

In [99]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [100]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [101]:
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


### 3.2.1 Handiling with NaN
En ocasiones, al hacer unstack pueden aparecer NaN values en los niveles que falten valores

In [102]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [103]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [104]:
s2

c    4
d    5
e    6
dtype: int64

In [105]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [106]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Podemos invertir incluyendo (o no) los NaN values

In [107]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [108]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

## 3.3 Pivoting
La función pivot puede considerarse como una forma de analizar el dataframe desde una perspectiva diferente. 

Nos permite filtrar datos y ordenarlos. Se utiliza para explorar la relación entre variables remodelando un DataFrame según las columnas e index que queramos.

Muy útil trabajando con Series Temporales.

In [109]:
Image(url= "https://miro.medium.com/max/875/1*FgARl-K9AnW3QBlYxeqZ9Q.jpeg", width =700)

In [110]:
df = pd.DataFrame(
    {
        "fruit": ["apple", "orange", "apple", "avocado", "orange"],
        "customer": ["ben", "alice", "ben", "josh", "steve"],
        "quantity": [1, 2, 3, 1, 2],
    }
)

df

,fruit,customer,quantity
0,apple,ben,1
1,orange,alice,2
2,apple,ben,3
3,avocado,josh,1
4,orange,steve,2


In [117]:
df.pivot_table(index="fruit", columns="customer", values="quantity", aggfunc=np.sum)

customer,alice,ben,josh,steve
fruit,,,,
apple,NaN,4.0,NaN,NaN
avocado,NaN,NaN,1.0,NaN
orange,2.0,NaN,NaN,2.0


Es lo mismo que:

In [112]:
df.groupby(['fruit', 'customer']).quantity.sum().unstack()

customer,alice,ben,josh,steve
fruit,,,,
apple,NaN,4.0,NaN,NaN
avocado,NaN,NaN,1.0,NaN
orange,2.0,NaN,NaN,2.0
